# Appendix - FillProba

In [1]:
import os
from working_dir import set_wd
set_wd()
os.getcwd()

'/home/tales/ds/kaggle/football-match-prediction'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.debug.maxToStringFields", 500) \
                            .config("spark.driver.memory", "14g") \
                            .appName("FillProba").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

22/05/15 16:04:54 WARN Utils: Your hostname, tales-samsung resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface wlxd03745e80dbf)
22/05/15 16:04:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/15 16:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import numpy as np
import pyspark.sql.functions as f
from pyspark.ml.feature import VectorAssembler
from src.ml.estimators import FillProbaEstimator
from src.dao import dao, dao_processed, dao_raw, dao_interim
from src.utils import dflib, stats, pretties, plot, plot_domain, palette

In [4]:
pretties.max_data_frame_columns()

# Loading Data

In [5]:
df_ttrain = dao_interim.load_train_train_data(spark)
# df_ttrain = dflib.sample(df_ttrain, 10000)

df_tvalid = dao_interim.load_train_valid_data(spark)
df_tvalid = dflib.sample(df_tvalid, 1000)

# FillProbaEstimator

In [6]:
def dummy_classifier(df, labels, output_col):
    df, df_na = df.randomSplit(weights=[0.8,0.2], seed=200)
    
    for label in labels:
        df = df.withColumn(label, f.lit(np.random.randint(low=1, high=100) / 100))

    feature_assembler_transformer = VectorAssembler(inputCols=labels,
                                                    outputCol=output_col)
    
    df_na = df_na.withColumn(output_col, f.lit(None))
    df = feature_assembler_transformer.transform(df).drop(*labels)
    
    return df.union(df_na)

In [7]:
# df_ttrain = dummy_classifier(df_ttrain, labels=["home", "draw", "away"], output_col="proba")

In [8]:
df_tvalid = dummy_classifier(df_tvalid, labels=["home", "draw", "away"], output_col="proba")

### Uniform Proba

In [9]:
fill_proba_transfomer = FillProbaEstimator(strategy="uniform_proba",
                                           labels=["home", "draw", "away"], 
                                           proba_vector_col="proba").fit(df_ttrain)
fill_proba_transfomer

FillProbaTransformer_e301b9a76cf4

In [10]:
df_ttrain_transformed = fill_proba_transfomer.transform(df_tvalid)

In [11]:
stats.freq(df_ttrain_transformed, "proba").show()

+--------------------+--------+--------+
|               proba|Absolute|Relative|
+--------------------+--------+--------+
|    [0.33,0.08,0.33]|     800|     0.8|
|[0.3333334,0.3333...|     200|     0.2|
+--------------------+--------+--------+



### Global Frequency

In [12]:
fill_proba_transfomer = FillProbaEstimator(strategy="global_frequency",
                                           labels=["home", "draw", "away"], 
                                           proba_vector_col="proba").fit(df_ttrain)
fill_proba_transfomer

FillProbaTransformer_599f82782aa7

In [13]:
df_ttrain_transformed = fill_proba_transfomer.transform(df_tvalid)

In [14]:
stats.freq(df_ttrain_transformed, "proba").show()

+--------------------+--------+--------+
|               proba|Absolute|Relative|
+--------------------+--------+--------+
|    [0.33,0.08,0.33]|     800|     0.8|
|[0.43568080484737...|     200|     0.2|
+--------------------+--------+--------+



### League Frequency

In [15]:
fill_proba_transfomer = FillProbaEstimator(strategy="league_frequency",
                                           labels=["home", "draw", "away"], 
                                           proba_vector_col="proba",
                                           strategy_b="uniform_proba").fit(df_ttrain)
fill_proba_transfomer

FillProbaTransformer_c393f628b982

In [16]:
df_tvalid_transformed = fill_proba_transfomer.transform(df_tvalid)

In [17]:
stats.freq(df_tvalid_transformed, "proba").show()

+--------------------+--------+--------+
|               proba|Absolute|Relative|
+--------------------+--------+--------+
|    [0.33,0.08,0.33]|     800|     0.8|
|[0.3333334,0.3333...|     200|     0.2|
+--------------------+--------+--------+



In [18]:
print("tvalid")
print(df_tvalid.count(), "df_tvalid")
print(df_tvalid_transformed.count(), "df_tvalid_transformed")
stats.freq(df_tvalid_transformed, "proba").show()

tvalid


1000 df_tvalid


1000 df_tvalid_transformed


+--------------------+--------+--------+
|               proba|Absolute|Relative|
+--------------------+--------+--------+
|    [0.33,0.08,0.33]|     800|     0.8|
|[0.3333334,0.3333...|     200|     0.2|
+--------------------+--------+--------+

